In [1]:
import datetime
import numpy as np
import talib as ta
import alpaca_trade_api as tradeapi
import time
import sys
sys.path.insert(0, '../')



In [2]:
from config_paper import key_id, secret_key
from cross_signal import *

In [3]:
#The url at the end is what makes a paper trading bot
base_url='https://paper-api.alpaca.markets'
api = tradeapi.REST(key_id, secret_key,base_url)

In [4]:
startdate = datetime.datetime.today().strftime("%Y-%m-%d")
startdate

'2020-04-28'

In [5]:
#make sure to import these packages
from pandas import DataFrame
import talib.abstract as tab
#abstract automatically extracts the columns and periods from the data itself, no input required



#feel free to add or subtract more indicators for any other strategy you want to use
def populateindicators(dataframe) -> DataFrame:
        #exponential moving averages
        dataframe['mfi'] = ta.MFI(dataframe['high'],dataframe['low'],dataframe['close'], np.asarray(dataframe['volume'], dtype='float'), timeperiod=14)
        dataframe['ultosc'] = ta.ULTOSC(dataframe['high'],dataframe['low'],dataframe['close'])
        
        dataframe['minusdi'] = tab.MINUS_DI(dataframe, timeperiod=14)
        dataframe['plusdi'] = tab.PLUS_DI(dataframe, timeperiod=14)
        
        dataframe['rsi'] = tab.RSI(dataframe, timeperiod=20, price='close')
        dataframe['adx'] = tab.ADX(dataframe)

        # required for graphing

        return dataframe

In [6]:
account = api.get_account()
print(account.status)
account.buying_power

ACTIVE


'97144.21'

In [7]:
with open('SP500_2020Q2SYM.txt', 'r') as f:
    symbols= f.read().split('\n')

In [8]:
barTimeframe = "1Min" # 1Min, 5Min, 15Min, 1H, 1D
assetsToTrade = [x.upper() for x in symbols]
positionSizing = 0.10

# Tracks position in list of symbols to download 
assetListLen = len(assetsToTrade)


In [ ]:
for minu in range(500):
#while trade=True:
        
    for i in range(assetListLen) :
        try:
            symbol = assetsToTrade[i]

            returned_data = api.polygon.historic_agg_v2(symbol, 1, 'minute',_from='2020-04-20', to=startdate,).df
            #api.get_barset(symbol,barTimeframe,limit=150).df


            # Calculated trading indicators
            #populate exponential moving averages
            data=populateindicators(returned_data) 


             # Create the buy signal here
            if data['adx'][-1] > 22 and \
            data['mfi'][-1] < 25 and \
            data['ultosc'][-1] < 25 and \
            data['rsi'][-1] < 33 and \
            data['minusdi'][-1] > 25 and \
            data['minusdi'][-1] > data['plusdi'][-1]:

                buyord=True
                assetlist=api.list_positions()
                for asset in assetlist:
                    if asset.symbol == symbol:
                        buyord=False
                        print(asset.symbol,asset.qty)
                if buyord==True:
                    #cashBalance = account.cash
                    cashBalance = account.cash

                    targetPositionSize = int(float(cashBalance) / (data['close'][-1] / positionSizing))
                    targetPositionSize = int(2000 / data['close'][-1]) # Calculates required position size

                    returned = api.submit_order(symbol,targetPositionSize,"buy","market","day") # Market order to open position
                    print('Buy ',returned)


            elif data['adx'][-1] > 22 and \
            data['mfi'][-1] > 63 and \
            data['rsi'][-1] > 75 and \
            data['plusdi'][-1] > 25 and \
            data['minusdi'][-1] < data['plusdi'][-1]:
                # Closes position 
                openPosition = api.get_position(symbol)

                returned = api.submit_order(symbol,int(openPosition.qty),"sell","market","day") # Market order to fully close position
                print('Sell',returned)
        
        except:
                print(symbol)
                
        #interval should be an integer, the number of seconds to wait
    time.sleep(20)
    
    #close when trading hours are done
    #Time=current
    #if currenttime > endtime:
        #trade = False

Buy  Order({   'asset_class': 'us_equity',
    'asset_id': '90e9c946-6323-49a4-be69-116f04fec288',
    'canceled_at': None,
    'client_order_id': '939ebdea-49b5-4df9-8798-4d8b31fc2a50',
    'created_at': '2020-04-28T15:57:56.173244Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'id': 'ae174863-4734-4e69-b06b-d1760cc43ddc',
    'legs': None,
    'limit_price': None,
    'order_class': '',
    'order_type': 'market',
    'qty': '23',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'buy',
    'status': 'accepted',
    'stop_price': None,
    'submitted_at': '2020-04-28T15:57:56.153538Z',
    'symbol': 'AEP',
    'time_in_force': 'day',
    'type': 'market',
    'updated_at': '2020-04-28T15:57:56.173244Z'})
BRKB
BFB
CMS 34
Buy  Order({   'asset_class': 'us_equity',
    'asset_id': 'c4bcff4e-c3b0-481a-a9eb-98600db2ef0a',
    'canceled_at': None

Buy  Order({   'asset_class': 'us_equity',
    'asset_id': '42f6d400-741b-4a18-ae61-73faae71e2cb',
    'canceled_at': None,
    'client_order_id': 'd13c527b-7a6c-431b-8b8b-cd9e9a85ec47',
    'created_at': '2020-04-28T16:21:04.348926Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'id': 'a311e173-1448-4b78-995b-d1f373ca7b42',
    'legs': None,
    'limit_price': None,
    'order_class': '',
    'order_type': 'market',
    'qty': '12',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'buy',
    'status': 'accepted',
    'stop_price': None,
    'submitted_at': '2020-04-28T16:21:04.319486Z',
    'symbol': 'DHR',
    'time_in_force': 'day',
    'type': 'market',
    'updated_at': '2020-04-28T16:21:04.348926Z'})
MKTK
RTN
UTX

BRKB
BFB
CINF 27
MKTK
RTN
UTX
